# Exercises for the course "Declarative Problem Solving Paradigms in AI"

In [1]:
# Setup (some dependency errors may appear, you can ingore them for now)
# !pip install cpmpy numpy --quiet

import cpmpy as cp
import time

**CPMpy documentation**: https://cpmpy.readthedocs.io/en/latest/index.html

## **Session 4: Advanced Modelling**

### Outline:
1. Viewpoints
2. Channelling
3. Auxiliary Variables
4. Implied Constraints
5. Extra Exercises

In [2]:
def solve_with_time(model: cp.Model) -> bool:
    """Solve model and time.

    - The `model.status()` time is just the time taken by the solver.
    - The elapsed time includes parsing of constraints of by cpmpy, etc.

    Args:
        model (cp.Model): The model.

    Returns:
        bool: Success.
    """

    start_time = time.time()

    solution_found = model.solve()

    end_time = time.time()
    elapsed_time = end_time - start_time

    print(model.status())
    print("Time elapsed: {:.4f} seconds".format(elapsed_time))

    return solution_found

## **1. Viewpoints**

In constraint programming, we often have multiple ways to represent the same problem. These representations, or "viewpoints," affect both how the CP solver explores the solution space and how easy it is to formulate the constraints. A carefully chosen viewpoint can:
- Reduce the complexity of the model, making it **faster** to solve
- Make the constraints **easier** to express
- **Implicitly** model constraints as part of the variables/domains definitions
- Improve **readability**

---

Consider the following problem:

We have $n$ pigeons and $m$ pigeonholes. Each pigeon must be assigned to one pigeonhole, but each pigeonhole can hold at most one pigeon.
Here, we consider $n=50$ pigeons and $m=60$ pigeonholes.

The output can be of the form:
```
Pigeon 1 goes to hole 1
Pigeon 2 goes to hole 7
...
Pigeon 50 goes to hole 53
```

Modelling this problem is quite straightforward. The focus is on constructing two models for this problem using **different viewpoints**.

### Viewpoint 1: Pigeons as variables

<details>
  <summary>Click to reveal hints</summary>

* We need a decision variable for each pigeon representing which hole it goes into.
* The [AllDifferent](https://cpmpy.readthedocs.io/en/latest/api/expressions/globalconstraints.html#cpmpy.expressions.globalconstraints.AllDifferent) constraint fits nicely when the pigeons are selected as decision variables.
</details>

In [3]:
num_pigeons = 50
num_pigeonholes = 60

# TODO: Decision variables
pigeons = cp.intvar(1, num_pigeonholes, shape=num_pigeons)

# Model
model_vp1 = cp.Model()

# TODO: Constraints
# Each pigeonhole can hold at most one pigeon, thus no two pigeons in the same hole
model_vp1 += cp.AllDifferent(pigeons)

if solve_with_time(model_vp1):
    print(pigeons.value())
else:
    print("No solution found.")

ExitStatus.OPTIMAL (0.038661 seconds)
Time elapsed: 0.0414 seconds
[50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27
 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10  9  8  7  6  5  4  3
  2  1]


### Viewpoint 2: Pigeonholes as variables

<details>
  <summary>Click to reveal hints</summary>

- Make sure that the domain includes also the option that the hole may be empty
- The constraint that each pigeon must be assigned to exactly one pigeonhole can be expressed in various ways, some of them are:
  - For each pigeon use `cp.sum` and ensure that it is equal to 1.
  - Use [GlobalCardinalityCount](https://cpmpy.readthedocs.io/en/latest/api/expressions/globalconstraints.html#cpmpy.expressions.globalconstraints.GlobalCardinalityCount)
  - Use [AllDifferentExcept0](https://cpmpy.readthedocs.io/en/latest/api/expressions/globalconstraints.html#cpmpy.expressions.globalconstraints.AllDifferentExcept0) and [Count](https://cpmpy.readthedocs.io/en/latest/api/expressions/globalfunctions.html#cpmpy.expressions.globalfunctions.Count)
</details>

In [4]:
# Decision variables
pigeonholes = cp.intvar(0, num_pigeons, shape=num_pigeonholes)

# Constraints
model_vp2 = cp.Model()

# TODO: Constraints
model_vp2 += cp.AllDifferentExcept0(pigeonholes)
model_vp2 += cp.Count(pigeonholes, val=0) == num_pigeonholes - num_pigeons

if solve_with_time(model_vp2):
    print(pigeonholes.value())
else:
    print("No solution found.")

ExitStatus.OPTIMAL (0.127636 seconds)
Time elapsed: 0.4461 seconds
[49 50 48 47 46 45 44 43 42 41 40 38 37 36 35 34 32 29 31 28 26 27 25 24
 17 16 11 10 30  6 23 13 18  8 20 33 12 39  9 15  7  5 22 14  4  3  2 19
  1 21  0  0  0  0  0  0  0  0  0  0]


**Observations:**
1. Which viewpoint was easier to model?
2. Which viewpoint was faster to solve?
3. Which constraints are implicitly covered in each viewpoint?

## **2. Channeling**

Some constraints are more naturally expressed in one viewpoint, while other constraints of the same model are maybe more naturally expressed in another viewpoint. In such cases, **channeling** is a useful technique that connects variables from different viewpoints within a single model. Some key benefits:

1. You can use the most intuitive viewpoint for each constraint.
2. The solver may deduce information across linked viewpoints, potentially speeding up the solving process.
3. The final models can become easier to read, maintain, and extend.

Channeling is achieved by adding constraints that ensure consistency between the decision variables of different viewpoints.

---

Consider the *Student Seating Problem* from the lecture:

You are tasked with finding an optimal seating arrangement for `nStudents = 15` students across `nTables = 5` tables, where there are `nChairs = 20` chairs in total. You are given a list of chair assignments for each table:
`Chairs = [1..4, 5..8, 9..12, 13..16, 17..20]`.
The goal is to assign **all** students to the chairs in such a way that each table has either at least half or no chairs occupied.

Create variables with two different viewpoints and try to model each constraint with the viewpoint that is easier. Before solving, it is necessary that the variables between the viewpoints are logically connected.

<details>
  <summary>Click to reveal hints</summary>

There are two high-level constraints that need to be satisfied:
1. From a student perspective: **All** students sit on a **different** chair
2. From a chair perspective: At each table, either **all chairs are empty** or **at least half are non-empty**.
</details>

In [5]:
# Data
n_students = 15
n_chairs = 20
n_tables = 5

# Chair indexes for each table
# [1..4, 5..8, 9..12, 13..16, 17..20]
chairs_per_table = n_chairs // n_tables
tables = [list(range(start, start + chairs_per_table))
          for start in range(1, n_chairs + 1, chairs_per_table)]

# Model
model = cp.Model()

# Viewpoint 1: Students as vars
students = cp.intvar(1, n_chairs, shape=n_students)  # chair of each student

# TODO: Add the constraint(s) that is (are) easier to express with vp1
model += cp.AllDifferent(students)

# Viewpoint 2: Chairs as vars
chairs = cp.intvar(0, n_students, shape=n_chairs)  # 0 means empty chair

# TODO: Add the constraint(s) that is (are) easier to express with vp2
# Constraint: Each table must have either at least half or no chairs occupied
for table in tables:
    chairs_at_table = [chairs[c - 1] for c in table]
    
    all_empty = cp.sum(chairs_at_table) == 0
    at_least_half_occupied = cp.Count(chairs_at_table, val=0) <= (len(table) // 2)
    
    model += all_empty | at_least_half_occupied

# Channeling: At this point we have modeled half of the constraints with the
# vars of vp1 and half with those of vp2. We need to merge the two viewpoints.

# TODO: Connect the students and chairs viewpoints
# For each student, the chair of this student from vp1 has to point to this student in vp2
model += [chairs[students[i] - 1] == (i + 1) for i in range(n_students)]

# TODO: Solve the model and print the solution
if solve_with_time(model):
    print("Solution found:")
    print(f"Chair of each student: {students.value()}")

    for i, table in enumerate(tables):
        print(f"Table {i+1}: ", end="")
        subset_chairs = [chairs[c - 1] for c in table]
        for chair in subset_chairs:
            if chair.value() != 0:
                print(f"{chair.value()} ", end="")
        print()

ExitStatus.OPTIMAL (0.020395 seconds)
Time elapsed: 0.0283 seconds
Solution found:
Chair of each student: [ 1  2  4 19  6 20 17  5  3  7 18 11 10  9  8]
Table 1: 1 2 9 3 
Table 2: 8 5 10 15 
Table 3: 14 13 12 14 
Table 4: 14 14 14 14 
Table 5: 7 11 4 6 


## **3. Auxiliary Variables**

Auxiliary variables can help in expressing complex constraints more naturally and clearly. They can be useful when establishing relationships between decision variables and additional properties or constraints, making it easier to formulate constraints. So, they are used:

* Because it is difficult to express the constraints in terms of the existing variables, or
* To allow the constraints to be expressed in a form that would propagate better.

**Car Sequencing**

Imagine an assembly line producing different types of cars, each with optional features (e.g. air-conditioning, sunroof etc.). Each station on the assembly line handles a specific feature and has a limited capacity, meaning it can only work on a certain percentage of the cars that pass through.

To prevent any station from being overwhelmed, cars must be arranged in a sequence so that the number of cars needing a particular feature never exceeds the station's capacity. For example, if a station can only handle half of the cars that pass by, then at most 1 out of every 2 cars in the sequence should require that feature.

A model of this problem is also available in the slides of lecture 5, page 23.
Try to model it without looking at the slides first.

In [6]:
?cp.GlobalCardinalityCount

Init signature: cp.GlobalCardinalityCount(vars, vals, occ, closed=False)

Docstring:     

GlobalCardinalityCount(vars,vals,occ): The number of occurrences of each value vals[i] in the list of variables vars must be equal to occ[i].

In [7]:
# Data
at_most = [1, 2, 2, 2, 1]  # The maximum number of times an option can be present in a group of consecutive timeslots (see "per_slots" below)
per_slots = [2, 3, 3, 5, 5]  # The number of consecutive timeslots (window) in which an option has a limit (see "at_most" above)

demand = [1, 1, 2, 2, 2, 2]  # The demand per type of car
requires = cp.cpm_array([
    [1, 0, 1, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 0, 0, 1],
    [0, 1, 0, 1, 0],
    [1, 0, 1, 0, 0],
    [1, 1, 0, 0, 0]])  # Which options (columns) are needed for each car type (rows).

n_cars = sum(demand)  # The amount of cars to sequence
n_options = len(at_most)  # The amount of different options
n_types = len(demand)  # The amount of different car types

# Decision Variables
sequence = cp.intvar(0, n_types - 1, shape=n_cars)  # TODO: The sequence of cars

# TODO: Define the auxiliary variables -> Which options are needed for each car in the sequence.
# Sequence of different options based on the car type
option = cp.intvar(0, 1, shape=(n_cars, n_options))

# Model
car_seq_model = cp.Model()

# TODO: The amount of each type of car in the sequence has to be equal to the demand for that type
car_seq_model += cp.GlobalCardinalityCount(sequence, vals=range(n_types), occ=demand)

# TODO: Make sure that the options in the option table correspond to those of the car type
for s in range(n_cars):
    car_seq_model += [option[s, o] == requires[sequence[s], o] for o in range(n_options)]

# TODO: Check that no more than "at most" car options are used per "per_slots" slots
for o in range(n_options):
    for s in range(n_cars - per_slots[o]):
        car_seq_model += (cp.sum(option[s : s + per_slots[o], o]) <= at_most[o])

# TODO: Solve and print the sequence of cars and which options are required per car
car_seq_model.solve()

print("Sequence:", end="  ")
for s in sequence.value():
    print(s, end=" ")
print()
print("------------------------------")

for idx, row in enumerate(option.value().T):
    row_int = [int(x) for x in row]
    print(f"Option {idx + 1}: ", end=" ")
    for x in row_int:
        print(x, end=" ")
    print()


Sequence:  0 2 5 1 5 3 4 2 3 4 
------------------------------
Option 1:  1 0 1 0 1 0 1 0 0 1 
Option 2:  0 1 1 0 1 1 0 1 1 0 
Option 3:  1 0 0 0 0 0 1 0 0 1 
Option 4:  1 0 0 1 0 1 0 0 1 0 
Option 5:  0 1 0 0 0 0 0 1 0 0 


For reference, here is a solution of this problem:

```
Sequence:  0 2 5 1 5 3 4 2 3 4
------------------------------
Option 1:  1 0 1 0 1 0 1 0 0 1
Option 2:  0 1 1 0 1 1 0 1 1 0
Option 3:  1 0 0 0 0 0 1 0 0 1
Option 4:  1 0 0 1 0 1 0 0 1 0
Option 5:  0 1 0 0 0 0 0 1 0 0
```

## **4. Implied Constraints**

As taken from the lecture slides:

Can we do something to make our model quicker?
During the search process, the solver will explore several infeasible parts of the search tree. Can we avoid that?
Although implied constraints are logically redundant, and do not change the set of solutions, they can be used to reduce the search effort of the solving process.

Consider the **Magic Series problem**, also seen at lecture 5, page 29:

The element at index i in I = 0..(n-1) is the number of occurrences of i, for an integer n. Create a model that finds a magic series for n = 100.

Print the time taken to solve this model.

In [8]:
n = 100

# TODO: Decision Variables
Magic = cp.intvar(0, n, shape=n)

# Model
model = cp.Model()

# TODO: Add the constraint
model += cp.GlobalCardinalityCount(Magic, vals=range(n), occ=Magic)

# TODO: Solve the model and print the solution (print the time taken)
solve_with_time(model)

print(Magic.value())

ExitStatus.OPTIMAL (3.0056100000000003 seconds)
Time elapsed: 4.0147 seconds
[96  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  0  0  0]


We can significantly improve solving time by adding implied constraints.

Add implied constraints (try it initially without looking at them in the slides), and compare the time taken to solve the new model with the above one.

In [9]:
# Implied constraints
model += n == cp.sum(Magic)
model += n == cp.sum([i * Magic[i] for i in range(n)])

solve_with_time(model)

print(Magic.value())

ExitStatus.OPTIMAL (0.061596000000000005 seconds)
Time elapsed: 1.1005 seconds
[96  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  0  0  0]


## **5. Extra Exercises**


### Auxiliary Variables: Orthogonal Latin Squares

Consider the [Orthogonal Latin Squares](https://en.wikipedia.org/wiki/Mutually_orthogonal_Latin_squares) problem:

A Latin square is an $ n \times n $ array filled with $n$ different symbols, each occurring exactly once in each row and exactly once in each column. In other words, a Latin Square is a square grid where:

* Each row contains all $n$ distinct symbols exactly once.
* Each column contains all $n$ distinct symbols exactly once.
* No symbol appears twice in the same row or column.

Two Latin squares are said to be orthogonal if, when placed on top of each other, each possible combination of their symbols appears exactly once.

**Instructions:**

You have a set of $ n $ integers (0..n-1), and your task is to construct two $ n \times n $ Latin squares that are orthogonal.

1. Define decision variables for the Latin squares.
2. Introduce auxiliary variables to represent each cell's pairing from both Latin squares.
3. Ensure that each pairing is unique across the entire grid.

In [10]:
n = 7  # Size of the Latin square, and number of symbols

# TODO: Decision variables
square1 = cp.intvar(0, n - 1, shape=(n, n), name="square1")
square2 = cp.intvar(0, n - 1, shape=(n, n), name="square2")

# TODO: Auxiliary variables to combine symbols from both squares
pairing = cp.intvar(0, n * n - 1, shape=(n, n), name="pairing")

# Model
model = cp.Model()

# Latin square constraints: all rows and columns should be all different for both squares
for i in range(n):
    model += [cp.AllDifferent(square1[i, :]), cp.AllDifferent(square1[:, i])]
    model += [cp.AllDifferent(square2[i, :]), cp.AllDifferent(square2[:, i])]

# Connect the pairing variables with the square variables
for i in range(n):
    for j in range(n):
        model += pairing[i, j] == n * square1[i, j] + square2[i, j]

# All pairings should be unique
model += cp.AllDifferent(pairing.flatten())

if model.solve():
    print("Solution found:")
    print("Square 1:")
    print(square1.value())
    print("Square 2:")
    print(square2.value())
    print("Pairings:")
    print(pairing.value())
else:
    print("No solution found.")

Solution found:
Square 1:
[[0 1 4 6 3 2 5]
 [1 2 3 5 0 6 4]
 [6 5 1 3 2 4 0]
 [5 4 2 0 6 3 1]
 [3 0 5 2 4 1 6]
 [2 6 0 4 1 5 3]
 [4 3 6 1 5 0 2]]
Square 2:
[[0 1 2 5 3 4 6]
 [4 2 6 1 5 3 0]
 [6 4 3 0 1 5 2]
 [3 6 0 4 2 1 5]
 [2 3 5 6 4 0 1]
 [5 0 1 3 6 2 4]
 [1 5 4 2 0 6 3]]
Pairings:
[[ 0  8 30 47 24 18 41]
 [11 16 27 36  5 45 28]
 [48 39 10 21 15 33  2]
 [38 34 14  4 44 22 12]
 [23  3 40 20 32  7 43]
 [19 42  1 31 13 37 25]
 [29 26 46  9 35  6 17]]


Now, to see the benefit of auxiliary variables, try to model the same problem but without using `pairings`. The orthogonality constraint needs to be formulated directly over the square variables. This is more complicated to model, and possibly also slower for the solver (check the run time).

In [11]:
n = 7  # Size of the Latin square, and number of symbols

# Decision variables for two Latin squares
square1 = cp.intvar(0, n - 1, shape=(n, n), name="square1")
square2 = cp.intvar(0, n - 1, shape=(n, n), name="square2")

# Model
model_no_aux = cp.Model()

# Latin square constraints: all rows and columns should be all different for both squares
for i in range(n):
    model_no_aux += [cp.AllDifferent(square1[i, :]), cp.AllDifferent(square1[:, i])]
    model_no_aux += [cp.AllDifferent(square2[i, :]), cp.AllDifferent(square2[:, i])]

# Orthogonality constraint: Ensure each combination is unique by pair-wise comparing all pairs
pairs = [(i, j) for i in range(n) for j in range(n)]
for idx1, (i1, j1) in enumerate(pairs):
    for idx2, (i2, j2) in enumerate(pairs):
        if idx1 < idx2:
            # Ensure that no two different positions result in the same pair (square1, square2)
            model_no_aux += (square1[i1, j1] != square1[i2, j2]) | (
                square2[i1, j1] != square2[i2, j2]
            )

if model_no_aux.solve():
    print("Solution found:")
    print("Square 1:")
    print(square1.value())
    print("Square 2:")
    print(square2.value())
else:
    print("No solution found.")

Solution found:
Square 1:
[[2 5 0 3 1 6 4]
 [3 2 4 0 6 1 5]
 [4 0 2 1 3 5 6]
 [1 4 6 2 5 3 0]
 [0 3 5 6 2 4 1]
 [6 1 3 5 4 0 2]
 [5 6 1 4 0 2 3]]
Square 2:
[[5 4 3 6 0 2 1]
 [2 3 0 1 6 4 5]
 [6 5 2 3 4 1 0]
 [1 2 4 0 3 5 6]
 [4 0 6 5 1 3 2]
 [3 6 1 2 5 0 4]
 [0 1 5 4 2 6 3]]


### Implied Constraints: Car Sequencing

Consider again the Car Sequencing problem. Add the implied constraints that are recommended in the slides of lecture 5, page 27.

For more details on the implied constraints of Car Sequencing, see this [article](https://www.researchgate.net/publication/220838036_Solving_the_Car_Sequencing_Problem_in_Constraint_Logic_Programming#fullTextFileContent), Pages 8-9.

In [12]:
# Implied constraints: Ensure we don’t stay too far below capacity
for o in range(n_options):
    # Total number of cars requiring this option
    M = sum([demand[car_type] * requires[car_type, o] for car_type in range(n_types)])

    # Total number of cars
    N = sum(demand)

    M_i = at_most[o]
    N_i = per_slots[o]

    k = 0
    while (M - k * M_i) >= 0:
        car_seq_model += cp.sum(option[: N - k * N_i, o]) >= M - k * M_i
        k += 1

solve_with_time(car_seq_model)

print("Sequence:", end="  ")
for s in sequence.value():
    print(s, end=" ")
print()
print("------------------------------")

for idx, row in enumerate(option.value().T):
    row_int = [int(x) for x in row]
    print(f"Option {idx + 1}: ", end=" ")
    for x in row_int:
        print(x, end=" ")
    print()

ExitStatus.OPTIMAL (0.011315 seconds)
Time elapsed: 0.0374 seconds
Sequence:  3 4 2 4 3 5 1 5 2 0 
------------------------------
Option 1:  0 1 0 1 0 1 0 1 0 1 
Option 2:  1 0 1 0 1 1 0 1 1 0 
Option 3:  0 1 0 1 0 0 0 0 0 1 
Option 4:  1 0 0 0 1 0 1 0 0 1 
Option 5:  0 0 1 0 0 0 0 0 1 0 
